In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

In [ ]:
df.head()

In [ ]:
#il faut trandormer les données en données numérique

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for each in df.columns:
    df[each] = le.fit_transform(df[each])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
#trés bien :pas de valeurs nulles , tous les données sont de type int64

In [ ]:
df['class'].value_counts()

In [ ]:
#quelques plot :

In [ ]:
plt.scatter(df['habitat'],df['class'])
plt.xlabel('habitat')
plt.ylabel('class')

In [ ]:
plt.scatter(df['population'],df['class'])
plt.xlabel('population')
plt.ylabel('class')

In [ ]:
sns.pairplot(df[['class', 'stalk-shape', 'veil-type', 'bruises']] )

In [ ]:

sns.heatmap(df.corr(), annot=True, fmt=".1f", linewidths=.5);

In [ ]:
#beaucoup des colonnes les résultat n'est pas clair

In [ ]:
columns = df.columns
columns

In [ ]:
columns1 = ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor','gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring','stalk-surface-below-ring']
columns2 = ['class','stalk-color-above-ring','stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number','ring-type', 'spore-print-color', 'population', 'habitat']

In [ ]:
df1 = df[columns1]
df2 = df[columns2]

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
sns.heatmap(df1.corr(), annot=True, fmt=".1f", linewidths=.5);

In [ ]:

sns.heatmap(df2.corr(), annot=True, fmt=".1f", linewidths=.5);

In [ ]:
#avec cette méthode on peut voir les colonnes qui influent sur la classe de mushrooms

In [ ]:
df['veil-type'].value_counts()

In [ ]:
df.drop('veil-type',axis=1,inplace=True)

In [ ]:
#d'aprés df1 la valeur de 'cap-color' n'a aucun influence sur la classe

In [ ]:
df.drop('cap-color',axis=1,inplace=True)

In [ ]:
x = df.drop('class',axis=1)
y = df['class']

In [ ]:
x_train , x_test ,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
x_train.shape

In [ ]:
#modéle 1

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=200,criterion='entropy',n_jobs=-1)

In [ ]:
rfc.fit(x_train,y_train)
pred = rfc.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

In [ ]:
#trés bien

In [ ]:
#je veut essayer juste le modéle de lgb

In [ ]:
import lightgbm as lgb
trn_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_test, label=y_test)

In [ ]:
weight = 10
param = { 'num_leaves': 80,
    'boost': 'gbdt',
    'learning_rate': 0.03,
    'metric':'binary_logloss',
    'scale_pos_weight': round(weight, 1),
    
 
    
    'objective': 'binary',
    'random_state': 42,
    
}

lgb_model = lgb.train(param, trn_data, 5000, valid_sets = [trn_data, val_data], 
                      verbose_eval=200, early_stopping_rounds=200)

In [ ]:
pred = lgb_model.predict(x_test,num_iteration=lgb_model.best_iteration)

In [ ]:
pred

In [ ]:
threshold = 0.5
pred  = (pred >=threshold)*1

In [ ]:
pred

In [ ]:
from sklearn.metrics import f1_score
f1_score(pred,y_test)

In [ ]:
accuracy_score(pred,y_test)